### Basic library imports

In [3]:
import os
import pandas as pd

### Read Dataset

In [4]:
DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

### Run Sanity check using src/sanity.py

In [3]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out.csv

Parsing successfull for file: ../dataset/sample_test_out.csv


In [4]:
!python sanity.py --test_filename ../dataset/sample_test.csv --output_filename ../dataset/sample_test_out_fail.csv

Error: Invalid unit [lbs] found in 6.75 lbs. Allowed units: {'millilitre', 'yard', 'imperial gallon', 'foot', 'millimetre', 'gallon', 'milligram', 'litre', 'cup', 'pound', 'microlitre', 'volt', 'centimetre', 'watt', 'metre', 'ton', 'ounce', 'pint', 'gram', 'quart', 'cubic foot', 'kilowatt', 'centilitre', 'millivolt', 'kilovolt', 'cubic inch', 'decilitre', 'fluid ounce', 'kilogram', 'inch', 'microgram'}


### Download images

In [5]:
# from utils import download_images
# download_images(sample_test['image_link'], '../images')

In [6]:
assert len(os.listdir('../images')) > 0

In [8]:
train.iloc[:4]

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram


In [10]:
test.iloc[:4]

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth


In [13]:
# !pip install pytesseract


In [8]:
import cv2 
from pytesseract import pytesseract 

# Defining paths to tesseract.exe 
# and the image we would be using 
path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
files = os.listdir('../images')[:3]
for image_path in files:
    image_path = f"../images/{image_path}"

    # Opening the image & storing it in an image object 
    img =cv2.imread(image_path)
    print(image_path)
    # Providing the tesseract executable 
    # location to pytesseract library 
    pytesseract.tesseract_cmd = path_to_tesseract 

    # Passing the image object to image_to_string() function 
    # This function will extract the text from the image 
    text = pytesseract.image_to_string(img) 

    # Displaying the extracted text 
    print(text[:-1])
    print("Done")


../images/41-NCxNuBxL.jpg
20CM

15C

Done
../images/417NJrPEk+L.jpg
Deodorizing module Cat litter shovel

‘Adsorb harmful substances and reduce odor Regardless of soil sand,

‘wood sand, crystal sand,
filter unused cat litter

Done
../images/417SThj+SrL.jpg
ated ine
— ev
assoc one ai
Netensspat =
Meco Coe 6 0

‘Nix Cuting Capac A HOAW2an

Done


In [11]:
# !pip install opencv-python

In [55]:
import cv2
from pytesseract import pytesseract as tess 
path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
tess.tesseract_cmd = path_to_tesseract
image_path = '../images/71Qk6hR9-WL.jpg'
img = cv2.imread(image_path)

text = tess.image_to_string(img)
print(text)
# Convert the image to gray scale
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

80 PLUS
CERTIFICATE

LONG CONNECTOR
CABLES

12CM FAN

500 WATTS



In [63]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
tess.image_to_string(gray)

'c=\n\n80 PLUS\nCERTIFICATE\n\nLONG CONNECTOR\nCABLES\n\n12CM FAN\n\n500 WATTS\n\n'

In [64]:
text = tess.image_to_string(gray)
text = text.replace('\n',' ')
text

'c=  80 PLUS CERTIFICATE  LONG CONNECTOR CABLES  12CM FAN  500 WATTS  '

In [82]:
import re

c = "\n\n80 PLUS\nCERTIFICATE\n\nLONG CONNECTOR\nCABLES\n\n12CM FAN\n\n500 WATTS\n\n"
c=c.replace("\n"," ")
# Regex to match quantities with their units, potentially across multiple lines
pattern = r"(\d+)\s*([A-Z]+)"

matches = re.findall(pattern, c)

# Print results
print(c,matches)


  80 PLUS CERTIFICATE  LONG CONNECTOR CABLES  12CM FAN  500 WATTS   [('80', 'PLUS'), ('12', 'CM'), ('500', 'WATTS')]


In [22]:


# Performing OTSU threshold
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# Specify structure shape and kernel size. 
# Kernel size increases or decreases the area 
# of the rectangle to be detected.
# A smaller value like (10, 10) will detect 
# each word instead of a sentence.
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# Applying dilation on the threshold image
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# Finding contours
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL, 
                                                 cv2.CHAIN_APPROX_NONE)

# Creating a copy of image
im2 = img.copy()

# A text file is created and flushed
file = open("recognized.txt", "w+")
file.write("")
file.close()

# Looping through the identified contours
# Then rectangular part is cropped and passed on
# to pytesseract for extracting text from it
# Extracted text is then written into the text file
for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)
    
    # Drawing a rectangle on copied image
    rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Cropping the text block for giving input to OCR
    cropped = im2[y:y + h, x:x + w]
    
    # Open the file in append mode
    file = open("recognized.txt", "a")
    
    # Apply OCR on the cropped image
    text = pytesseract.image_to_string(cropped)
    
    # Appending the text into file
    file.write(text)
    file.write("\n")
    
    # Close the file
    file.close()

# This code is modified by Susobhan Akhuli


In [27]:
inverted_image = cv2.bitwise_not(img)
print(tess.image_to_string(inverted_image))

Blade Diameter 505m
Rated Vols: nav
Frequency: sou
Rated Input Power 2100w

foLoad Speed 500

0° HID W330

Max Curing Capacity: 6 Ho) Wisoum

‘Mx Cuting Capacity A HO)AW20an




In [47]:
thres, img_bw = cv2.threshold(gray,127,200,cv2.THRESH_BINARY)
cv2.imwrite('temp/img_bw.jpg',img_bw)
print(tess.image_to_string(img_bw))

Professional tools

Blade Disaster 3030
Raed Vota. ov
Frequinsy son
RuedteputPoxee ai

Load Speed 30pm,
‘ax Coreng Capacioe Son Won
‘Max Curing Capacity. 6 KAO W160

‘Mar Cutng Capacity:

45° H60 Wm




In [80]:
# !pip install paddleocr

In [54]:

# from paddleocr import PaddleOCR
# ocr = PaddleOCR(use_angle_cls=True, lang='en')
# result = ocr.ocr(img)